In [1]:
#Importing the necessary Libraries

import random
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Activation
from tensorflow.keras.optimizers import RMSprop

In [2]:
import tensorflow as tf

filepath = tf.keras.utils.get_file('shakespear.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

text = open(filepath, 'rb').read().decode(encoding = 'utf-8').lower()

Streaming output truncated to the last 5000 lines.
my lord, 'tis but begun.

sly:
'tis a very excellent piece of work, madam lady:
would 'twere done!

petruchio:
verona, for a while i take my leave,
to see my friends in padua, but of all
my best beloved and approved friend,
hortensio; and i trow this is his house.
here, sirrah grumio; knock, i say.

grumio:
knock, sir! whom should i knock? is there man has
rebused your worship?

petruchio:
villain, i say, knock me here soundly.

grumio:
knock you here, sir! why, sir, what am i, sir, that
i should knock you here, sir?

petruchio:
villain, i say, knock me at this gate
and rap me well, or i'll knock your knave's pate.

grumio:
my master is grown quarrelsome. i should knock
you first,
and then i know after who comes by the worst.

petruchio:
will it not be?
faith, sirrah, an you'll not knock, i'll ring it;
i'll try how you can sol, fa, and sing it.

grumio:
help, masters, help! my master is mad.

petruchio:
now, knock when i bid you, sirra

In [3]:
# #Selecting the characters that will be used in training to reduce training time

# text = text[100000:1000000]

In [4]:
#Specifying the characters

characters = sorted(set(text)) #Set identifies the unique characters, sorted arranges them in alphabetical order

In [5]:
#Converting the text to Numerical format and vice versa using dictionaries

char_to_index = dict((c, i) for i, c in enumerate(characters))

#i.e {'a':1, 'b':2, 'c':3 etc}

index_to_char = dict((i, c) for i, c in enumerate(characters))

#i.e {1:'a', 2:'b', 3:'c' etc}

In [6]:
#Character prediction

SEQ_LEN = 40 # We will predict the next character based on the previous 40 characters
STEP_SIZE = 3

#Creating an empty list of the sentences and next charcters

sentences = []
next_characters = []

for i in range(0, len(text)- SEQ_LEN, STEP_SIZE):
  sentences.append(text[i: i+SEQ_LEN])
  next_characters.append(text[i+SEQ_LEN])

In [7]:
#Converting the sentences and next characters generted into numpy arrays

x = np.zeros((len(sentences), SEQ_LEN, len(characters)), dtype = bool)
y = np.zeros((len(sentences), len(characters)), dtype = bool)

In [8]:
#Filling the two arrays using for loops

for i, sentence in enumerate(sentences):
  for t, character in enumerate(sentence):
      x[i,t,char_to_index[character]] = 1
  y[i, char_to_index[next_characters[i]]] = 1

In [9]:
#Building the Recurrent Neural Network

model = Sequential()

model.add(LSTM(128, input_shape = (SEQ_LEN, len(characters))))
model.add(Dense(len(characters)))
model.add(Activation('softmax'))

model.compile(loss = 'categorical_crossentropy', optimizer = RMSprop(learning_rate = 0.01))

model.fit(x, y, batch_size = 256, epochs = 20)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/20
1453/1453 ━━━━━━━━━━━━━━━━━━━━ 11s 6ms/step - loss: 2.2269
Epoch 2/20
1453/1453 ━━━━━━━━━━━━━━━━━━━━ 11s 7ms/step - loss: 1.6074
Epoch 3/20
1453/1453 ━━━━━━━━━━━━━━━━━━━━ 19s 7ms/step - loss: 1.5094
Epoch 4/20
1453/1453 ━━━━━━━━━━━━━━━━━━━━ 9s 6ms/step - loss: 1.4572
Epoch 5/20
1453/1453 ━━━━━━━━━━━━━━━━━━━━ 10s 6ms/step - loss: 1.4290
Epoch 6/20
1453/1453 ━━━━━━━━━━━━━━━━━━━━ 9s 6ms/step - loss: 1.4032
Epoch 7/20
1453/1453 ━━━━━━━━━━━━━━━━━━━━ 10s 6ms/step - loss: 1.3886
Epoch 8/20
1453/1453 ━━━━━━━━━━━━━━━━━━━━ 10s 7ms/step - loss: 1.3742
Epoch 9/20
1453/1453 ━━━━━━━━━━━━━━━━━━━━ 10s 6ms/step - loss: 1.3609
Epoch 10/20
1453/1453 ━━━━━━━━━━━━━━━━━━━━ 11s 6ms/step - loss: 1.3506
Epoch 11/20
1453/1453 ━━━━━━━━━━━━━━━━━━━━ 9s 6ms/step - loss: 1.3451
Epoch 12/20
1453/1453 ━━━━━━━━━━━━━━━━━━━━ 10s 7ms/step - loss: 1.3400
Epoch 13/20
1453/1453 ━━━━━━━━━━━━━━━━━━━━ 10s 6ms/step - loss: 1.3331
Epoch 14/20
1453/1453 ━━━━━━━━━━━━━━━━━━━━ 10s 6ms/step - loss: 1.3272
Epoch 15/20
1453/1

In [10]:
#Saving the model

model.save('text_generator.keras')

In [11]:
#Loading the model

model = tf.keras.models.load_model('generator.keras')

In [12]:
#making predictions of the next character and converting them to text

def sample(preds, temperature = 1.0):
  preds = np.asarray(preds).astype('float64')
  preds = np.log(preds)/temperature
  exp_preds = np.exp(preds)
  preds = exp_preds/np.sum(exp_preds)
  probas = np.random.multinomial(1,preds,1)
  return np.argmax(probas)

In [13]:
#Function to generate the text

def generate_text(length, temperature):
  start_index = random.randint(0, len(text) - SEQ_LEN - 1)
  generated = ''
  sentence = text[start_index: start_index +SEQ_LEN]
  generated += sentence
  for i in range(length):
    x = np.zeros((1, SEQ_LEN, len(characters)))
    for t, character in enumerate(sentence):
        x[0,t, char_to_index[character]] = 1

    predictions = model.predict(x, verbose = 0)[0]
    next_index = sample(predictions, temperature)
    next_character = index_to_char[next_index] # Converting the index to the next character


    generated = generated+next_character
    sentence = sentence[1:] + next_character
  return generated

print('-----------0.2-------------')
print(generate_text(300, 0.2))
print('')

print('-----------0.4-------------')
print(generate_text(300, 0.4))
print('')

print('-----------0.6-------------')
print(generate_text(300, 0.6))
print('')

print('-----------0.8-------------')
print(generate_text(300, 0.8))
print('')

print('-----------1.0-------------')
print(generate_text(300, 1.0))
print('')

-----------0.2-------------
ne eyes.
therefore, no more but this: he hath the bears,
and the sensenion of the country's blood
and the state and the prince since it is a beat of the court
of the head that is the contratted of the heads
and the peace and the man to the country
of the state and she a presence and the state
of the counterkelse that is the senseness of t

-----------0.4-------------
s daughter fairer than she is,
she may make him as the rose of the villain.

king richard iii:
the counsels of the court, the strives here is a man are the fair
women out the counsels of his condick to be the counsell haste thee,
that they are by the manifes of my though here all the accused,
and thou shalt be thou art the for the land
th

-----------0.6-------------
nge his spots: take but my shame.
and i will be well for the issue and the country:
and that we shall be done on the friends.

aufidius:
my lord:
the world of my fire and bread the stirned be both.

lady anne:
well, infemned her lor